# Menjalankan QA Tanpa Intermediate Task - Transfer Learning

# Import semua module

In [1]:
#!pip install datasets
#!pip install transformers
#!pip install tensorboard
#!pip install evaluate
#!pip install git+https://github.com/IndoNLP/nusa-crowd.git@release_exp

In [2]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS'] = 'true'

In [3]:
!pip install -r requirements.txt

  Cloning https://github.com/IndoNLP/nusa-crowd.git (to revision 7748513d20331e72f9969f94f5d43c7f2d4a59a5) to /tmp/pip-install-i60hyohe/nusacrowd_b8fd39892f6a409c99b4227a9c4ffcb7
  Running command git clone --filter=blob:none -q https://github.com/IndoNLP/nusa-crowd.git /tmp/pip-install-i60hyohe/nusacrowd_b8fd39892f6a409c99b4227a9c4ffcb7
  Running command git rev-parse -q --verify 'sha^7748513d20331e72f9969f94f5d43c7f2d4a59a5'
  Running command git fetch -q https://github.com/IndoNLP/nusa-crowd.git 7748513d20331e72f9969f94f5d43c7f2d4a59a5
  Running command git checkout -q 7748513d20331e72f9969f94f5d43c7f2d4a59a5
  Resolved https://github.com/IndoNLP/nusa-crowd.git to commit 7748513d20331e72f9969f94f5d43c7f2d4a59a5
  Preparing metadata (setup.py) ... done
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [4]:
# Melihat GPU yang tersedia dan penggunaannya.
!nvidia-smi

Fri Mar 10 10:50:34 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.126.02   Driver Version: 418.126.02   CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:06:00.0 Off |                    0 |
| N/A   38C    P0    44W / 300W |     11MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   47C    P0   253W / 300W |  24686MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [5]:
# Memilih GPU yang akan digunakan (contohnya: GPU #7)
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [6]:
import transformers
import evaluate
import torch
import operator
import ast
import json
import re
import sys

import numpy as np
import pandas as pd
import torch.nn as nn

from multiprocessing import cpu_count
from evaluate import load
from nusacrowd import NusantaraConfigHelper
from torch.utils.data import DataLoader
from datetime import datetime
from huggingface_hub import notebook_login
from tqdm import tqdm

from datasets import (
    load_dataset, 
    load_from_disk,
    Dataset,
    DatasetDict
)
from transformers import (
    BigBirdTokenizerFast,
    BigBirdForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    BertForSequenceClassification,
    BertForQuestionAnswering,
    AutoModel, 
    BertTokenizerFast,
    AutoTokenizer, 
    AutoModel, 
    BertTokenizer, 
    BertForPreTraining,
    AutoModelForSequenceClassification,
    AutoModelForQuestionAnswering,
    EvalPrediction,
)

# Definisikan hyperparameter

In [7]:
MODEL_NAME = "indolem/indobert-base-uncased"
#MODEL_NAME = "afaji/fine-tuned-IndoNLI-Translated-with-indobert-base-uncased"
SEED = 42
EPOCH = 1
BATCH_SIZE = 32
GRADIENT_ACCUMULATION = 4
LEARNING_RATE = 1e-5
MAX_LENGTH = 400
STRIDE = 100
LOGGING_STEPS = 50
WARMUP_RATIO = 0.06
WEIGHT_DECAY = 0.01
# Untuk mempercepat training, saya ubah SAMPLE menjadi 100.
# Bila mau menggunakan keseluruhan data, gunakan: 
SAMPLE = sys.maxsize
# SAMPLE = 10

# Import dataset QAS

In [8]:
!pip install git+https://github.com/IndoNLP/nusa-crowd.git

  Cloning https://github.com/IndoNLP/nusa-crowd.git to /tmp/pip-req-build-loyehmwa
  Running command git clone --filter=blob:none -q https://github.com/IndoNLP/nusa-crowd.git /tmp/pip-req-build-loyehmwa
  Resolved https://github.com/IndoNLP/nusa-crowd.git to commit bea0aedb653c65d2dbc65a5f7b6950bd2cad274d
  Preparing metadata (setup.py) ... done
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Definisikan tokenizer

In [10]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Definisikan fungsi pre-processnya

In [11]:
conhelps = NusantaraConfigHelper()
data_qas_id = conhelps.filtered(lambda x: 'squad_id' in x.dataset_name)[0].load_dataset()

df_train = pd.DataFrame(data_qas_id['train'])
df_validation = pd.DataFrame(data_qas_id['validation'])

cols = ['context', 'question', 'answer']
new_df_val = pd.DataFrame(columns=cols)

for i in tqdm(range(len(df_validation['context']))):
    new_df_val = new_df_val.append({'context': df_validation["context"][i], 
                                    'question': df_validation["question"][i], 
                                    'answer': {"text": eval(df_validation["answer"][i][0])['text'], 
                                    "answer_start": eval(df_validation["answer"][i][0])['answer_start'], 
                                    "answer_end": eval(df_validation["answer"][i][0])['answer_end']}}, 
                                ignore_index=True)

cols = ['context', 'question', 'answer']
new_df_train = pd.DataFrame(columns=cols)

for i in tqdm(range(len(df_train['context']))):
    new_df_train = new_df_train.append({'context': df_train["context"][i], 
                                    'question': df_train["question"][i], 
                                    'answer': {"text": eval(df_train["answer"][i][0])['text'], 
                                    "answer_start": eval(df_train["answer"][i][0])['answer_start'], 
                                    "answer_end": eval(df_train["answer"][i][0])['answer_end']}}, 
                                ignore_index=True)

train_dataset = Dataset.from_dict(new_df_train)
validation_dataset = Dataset.from_dict(new_df_val)

data_qas_id = DatasetDict({"train": train_dataset, "validation": validation_dataset})

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████| 130319/130319 [08:53<00:00, 244.28it/s]


In [12]:
def rindex(lst, value, operator=operator):
      return len(lst) - operator.indexOf(reversed(lst), value) - 1

def preprocess_function_qa(examples, tokenizer, MAX_LENGTH=MAX_LENGTH, STRIDE=STRIDE, rindex=rindex, operator=operator):
    examples["question"] = [q.lstrip() for q in examples["question"]]
    examples["context"] = [c.lstrip() for c in examples["context"]]

    tokenized_examples = tokenizer(
      examples['question'],
      examples['context'],
      truncation=True,
      max_length = MAX_LENGTH,
      stride=STRIDE,
      return_overflowing_tokens=True,
      return_offsets_mapping=True,
      padding="max_length",
      return_tensors='np'
    )

    tokenized_examples['start_positions'] = []
    tokenized_examples['end_positions'] = []

    for seq_idx in range(len(tokenized_examples['input_ids'])):
        seq_ids = tokenized_examples.sequence_ids(seq_idx)
        offset_mappings = tokenized_examples['offset_mapping'][seq_idx]

        cur_example_idx = tokenized_examples['overflow_to_sample_mapping'][seq_idx]

        #answer = examples['answer'][seq_idx][0]
        answer = examples['answer'][cur_example_idx]
        answer = eval(str(answer))
        #answer_text = answer['text'][0]
        answer_start = answer['answer_start']
        #answer_end = answer_start + len(answer_text)
        answer_end = answer['answer_end']

        context_pos_start = seq_ids.index(1)
        context_pos_end = rindex(seq_ids, 1, operator)

        s = e = 0
        if (offset_mappings[context_pos_start][0] <= answer_start and
            offset_mappings[context_pos_end][1] >= answer_end):
          i = context_pos_start
          while offset_mappings[i][0] < answer_start:
            i += 1
          if offset_mappings[i][0] == answer_start:
            s = i
          else:
            s = i - 1

          j = context_pos_end
          while offset_mappings[j][1] > answer_end:
            j -= 1      
          if offset_mappings[j][1] == answer_end:
            e = j
          else:
            e = j + 1

        tokenized_examples['start_positions'].append(s)
        tokenized_examples['end_positions'].append(e)
    return tokenized_examples

# Mulai tokenisasi dan pre-process

In [13]:
tokenized_data_qas_id = data_qas_id.map(
    preprocess_function_qa,
    batched=True,
    remove_columns=data_qas_id['train'].column_names,
    num_proc=1,
    fn_kwargs={'tokenizer': tokenizer, 'MAX_LENGTH': MAX_LENGTH, 'STRIDE': STRIDE, 'rindex': rindex, 'operator': operator}
)

  0%|          | 0/131 [00:00<?, ?ba/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

In [14]:
tokenized_data_qas_id = tokenized_data_qas_id.remove_columns(["offset_mapping", 
                                            "overflow_to_sample_mapping"])

In [15]:
tokenized_data_qas_id.set_format("torch", columns=["input_ids", "token_type_ids"], output_all_columns=True, device=device)

In [16]:
tokenized_data_qas_id_train = Dataset.from_dict(tokenized_data_qas_id["train"][:SAMPLE])
tokenized_data_qas_id_validation = Dataset.from_dict(tokenized_data_qas_id["validation"][:SAMPLE])

# Mendefinisikan argumen (dataops) untuk training nanti

In [17]:
TIME_NOW = str(datetime.now()).replace(":", "-").replace(" ", "_").replace(".", "_")
QA = './results/alur2-idk-mrc'
CHECKPOINT_DIR = f'{QA}-{TIME_NOW}/checkpoint/'
MODEL_DIR = f'{QA}-{TIME_NOW}/model/'
OUTPUT_DIR = f'{QA}-{TIME_NOW}/output/'
ACCURACY_DIR = f'{QA}-{TIME_NOW}/accuracy/'

# Mendefinisikan Training Arguments untuk train

In [18]:
training_args_qa = TrainingArguments(
    
    # Checkpoint
    output_dir=CHECKPOINT_DIR,
    save_strategy='epoch',
    save_total_limit=EPOCH,
    
    # Log
    report_to='tensorboard',
    logging_strategy='steps',
    logging_first_step=True,
    logging_steps=LOGGING_STEPS,
    
    # Train
    num_train_epochs=EPOCH,
    weight_decay=WEIGHT_DECAY,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION,
    learning_rate=LEARNING_RATE,
    warmup_ratio=WARMUP_RATIO,
    bf16=False,
    dataloader_num_workers=cpu_count(),
    
    # Miscellaneous
    evaluation_strategy='epoch',
    seed=SEED,
)

# Pendefinisian model Question Answering

In [19]:
model_qa = BertForQuestionAnswering.from_pretrained(MODEL_NAME)

Some weights of the model checkpoint at indolem/indobert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at indolem/indobert-base-uncased and a

In [20]:
model_qa = model_qa.to(device)

# Melakukan pengumpulan data dengan padding

In [21]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Mulai training untuk fine-tune SQUAD diatas IndoBERT

In [22]:
import string
import collections

# # Melakukan evaluasi dari prediksi
def normalize_text(s):
    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)
    def white_space_fix(text):
        return " ".join(text.split())
    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)
    def lower(text):
        return text.lower()
    return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_f1_prec_rec(pred, gold):
    pred_tokens = normalize_text(pred).split() # True positive + False positive = Untuk precision
    gold_tokens = normalize_text(gold).split() # True positive + False negatives = Untuk recall
    common = collections.Counter(pred_tokens) & collections.Counter(gold_tokens)
    num_same = sum(common.values()) # True positive

    if len(gold_tokens) == 0 or len(pred_tokens) == 0: 
        var = int(gold_tokens == pred_tokens)
        return var, var, var

    if num_same == 0:
        return 0, 0, 0

    precision = 1.0 * num_same / len(pred_tokens)
    recall = 1.0 * num_same / len(gold_tokens)
    f1 = (2.0 * precision * recall) / (precision + recall)

    return f1, precision, recall

def compute_metrics(predict_result):
    predictions_idx = np.argmax(predict_result.predictions, axis=2)
    denominator = len(predictions_idx[0])
    label_array = np.asarray(predict_result.label_ids)
    total_correct = 0
    f1_array = []
    precision_array = []
    recall_array = []

    for i in range(len(predict_result.predictions[0])):
        start_pred_idx = predictions_idx[0][i]
        end_pred_idx = predictions_idx[1][i] + 1
        start_gold_idx = label_array[0][i]
        end_gold_idx = label_array[1][i] + 1

        pred_text = tokenizer.decode(tokenized_data_qas_id_validation[i]['input_ids']
                                    [start_pred_idx: end_pred_idx])
        gold_text = tokenizer.decode(tokenized_data_qas_id_validation[i]['input_ids']
                                    [start_gold_idx: end_gold_idx])

        if pred_text == gold_text:
            total_correct += 1

        f1, precision, recall = compute_f1_prec_rec(pred=pred_text, gold=gold_text)

        f1_array.append(f1)
        precision_array.append(precision)
        recall_array.append(recall)

    exact_match = ((total_correct / denominator) * 100.0)
    final_f1 = np.mean(f1_array) * 100.0
    final_precision = np.mean(precision_array) * 100.0
    final_recall = np.mean(recall_array) * 100.0

    return {'exact_match': exact_match, 'f1': final_f1, 'precision': final_precision, 'recall': final_recall}

In [23]:
trainer_qa = Trainer(
    model=model_qa.to(device),
    args=training_args_qa,
    train_dataset=tokenized_data_qas_id_train,
    eval_dataset=tokenized_data_qas_id_validation,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [24]:
trainer_qa.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 131028
  Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 4
  Total optimization steps = 1023
  Number of trainable parameters = 109969154


Epoch,Training Loss,Validation Loss,Exact Match,F1,Precision,Recall
0,1.949300,1.928556,37.802545,52.933379,54.562138,60.610682


***** Running Evaluation *****
  Num examples = 12023
  Batch size = 8
Saving model checkpoint to ./results/alur2-idk-mrc-2023-03-10_11-03-04_966438/checkpoint/checkpoint-1023
Configuration saved in ./results/alur2-idk-mrc-2023-03-10_11-03-04_966438/checkpoint/checkpoint-1023/config.json
Model weights saved in ./results/alur2-idk-mrc-2023-03-10_11-03-04_966438/checkpoint/checkpoint-1023/pytorch_model.bin
tokenizer config file saved in ./results/alur2-idk-mrc-2023-03-10_11-03-04_966438/checkpoint/checkpoint-1023/tokenizer_config.json
Special tokens file saved in ./results/alur2-idk-mrc-2023-03-10_11-03-04_966438/checkpoint/checkpoint-1023/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1023, training_loss=2.4144637153528534, metrics={'train_runtime': 2937.8283, 'train_samples_per_second': 44.6, 'train_steps_per_second': 0.348, 'total_flos': 2.67306582140928e+16, 'train_loss': 2.4144637153528534, 'epoch': 1.0})

# Menyimpan model Question Answering

In [25]:
trainer_qa.save_model(MODEL_DIR)

Saving model checkpoint to ./results/alur2-idk-mrc-2023-03-10_11-03-04_966438/model/
Configuration saved in ./results/alur2-idk-mrc-2023-03-10_11-03-04_966438/model/config.json
Model weights saved in ./results/alur2-idk-mrc-2023-03-10_11-03-04_966438/model/pytorch_model.bin
tokenizer config file saved in ./results/alur2-idk-mrc-2023-03-10_11-03-04_966438/model/tokenizer_config.json
Special tokens file saved in ./results/alur2-idk-mrc-2023-03-10_11-03-04_966438/model/special_tokens_map.json


# Melakukan prediksi dari model

In [26]:
predict_result = trainer_qa.predict(tokenized_data_qas_id_validation)
predict_result

***** Running Prediction *****
  Num examples = 12023
  Batch size = 8


PredictionOutput(predictions=(array([[ 2.3400595, -4.9847035, -5.3231263, ..., -7.67994  , -7.6520615,
        -7.993825 ],
       [ 2.0694578, -4.7709394, -5.695021 , ..., -7.479656 , -7.634318 ,
        -7.8908734],
       [ 2.2772768, -6.0087466, -6.756085 , ..., -7.807231 , -7.785742 ,
        -7.6126475],
       ...,
       [ 2.2395368, -1.4549687, -5.6119843, ..., -8.549288 , -8.349885 ,
        -7.8284698],
       [ 2.460146 , -3.6223714, -7.33974  , ..., -7.147509 , -7.0236773,
        -7.917438 ],
       [ 2.387605 , -3.9669983, -6.922913 , ..., -6.703164 , -7.771115 ,
        -7.5800157]], dtype=float32), array([[ 1.8328297, -5.4852195, -5.9295864, ..., -8.226759 , -8.231143 ,
        -8.035612 ],
       [ 1.8535712, -6.1044908, -6.478697 , ..., -8.087011 , -7.573409 ,
        -6.808301 ],
       [ 1.944047 , -6.583405 , -6.6930447, ..., -8.277608 , -8.29374  ,
        -8.166326 ],
       ...,
       [ 1.9341775, -3.4373777, -3.8312209, ..., -5.731249 , -6.1923976,
        -7

In [27]:
os.makedirs(os.path.dirname(OUTPUT_DIR), exist_ok=True)
with open(f'{OUTPUT_DIR}/output.txt', "w") as f:
  f.write(str(predict_result))
  f.close()

# Melakukan evaluasi dari prediksi

In [28]:
accuracy_result = compute_metrics(predict_result)
accuracy_result

{'exact_match': 37.80254512184979,
 'f1': 52.93337928997236,
 'precision': 54.56213837306034,
 'recall': 60.61068180559022}

In [29]:
os.makedirs(os.path.dirname(ACCURACY_DIR), exist_ok=True)
with open(f'{ACCURACY_DIR}/accuracy.txt', "w") as f:
  f.write(str(accuracy_result))
  f.close()

## Coba Alur 2

In [30]:
x = np.arange(60).reshape(3, 4, 5)
x

array([[[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19]],

       [[20, 21, 22, 23, 24],
        [25, 26, 27, 28, 29],
        [30, 31, 32, 33, 34],
        [35, 36, 37, 38, 39]],

       [[40, 41, 42, 43, 44],
        [45, 46, 47, 48, 49],
        [50, 51, 52, 53, 54],
        [55, 56, 57, 58, 59]]])

In [31]:
x[0:3, 0:4, -2]

array([[ 3,  8, 13, 18],
       [23, 28, 33, 38],
       [43, 48, 53, 58]])

In [32]:
len(predict_result.predictions)

2

In [33]:
x.shape

(3, 4, 5)

In [34]:
x = np.array([[0, 3], [2, 2], [4, 2]])
print(x.shape)
print(np.argsort(x)[-1][-1])
print(np.argmax(x))

(3, 2)
0
4


In [35]:
x = np.argmax(predict_result.predictions, axis=2)
print(x.shape)
x

(2, 12023)


array([[50, 32, 75, ..., 29, 54, 96],
       [50, 39, 75, ..., 30, 55, 97]])

In [36]:
x = np.argsort(predict_result.predictions, axis=2)[:,:, -0]
print(x.shape)
x

(2, 12023)


array([[238, 236, 196, ..., 262, 260, 259],
       [224, 166, 225, ..., 327, 240, 356]])

In [37]:
from tqdm import tqdm

def create_qas_dataframe(predict_result=predict_result, index_largest=1):
    predictions_idx = np.argsort(predict_result.predictions, axis=2)[:, :, index_largest * -1]
    #predictions_idx = np.argmax(predict_result.predictions, axis=2)
    label_array = np.asarray(predict_result.label_ids)
    question_decoded = []
    context_decoded = []
    pred_answer_decoded = []
    gold_answer_decoded = []
    
    for i in tqdm(range(len(predict_result.predictions[0]))):
        start_pred_idx = predictions_idx[0][i]
        end_pred_idx = predictions_idx[1][i] + 1
        pred_answer = tokenizer.decode(tokenized_data_qas_id_validation[i]['input_ids']
                                       [start_pred_idx: end_pred_idx], skip_special_tokens=True)
        pred_answer_decoded.append(pred_answer)
        
        start_gold_idx = label_array[0][i]
        end_gold_idx = label_array[1][i] + 1
        gold_answer = tokenizer.decode(tokenized_data_qas_id_validation[i]['input_ids']
                                       [start_gold_idx: end_gold_idx], skip_special_tokens=True)
        gold_answer_decoded.append(gold_answer)
        
        question = []
        context = []
         
        for j in range(len(tokenized_data_qas_id_validation[i]['token_type_ids'])):
            if tokenized_data_qas_id_validation[i]['token_type_ids'][j] == 0:
                question.append(tokenized_data_qas_id_validation[i]['input_ids'][j])
            else:
                context.append(tokenized_data_qas_id_validation[i]['input_ids'][j])

        question_decoded.append(tokenizer.decode(question, skip_special_tokens=True))
        context_decoded.append(tokenizer.decode(context, skip_special_tokens=True))
    
    qas_df = pd.DataFrame({'Context': context_decoded, 
                           'Question': question_decoded, 
                           'Prediction Answer': pred_answer_decoded,
                          'Gold Answer': gold_answer_decoded})
                      
    return qas_df

In [ ]:
qas_df = create_qas_dataframe(predict_result)
qas_df

 67%|████████████████████████████████████████████████████▎                         | 8059/12023 [54:54<33:07,  1.99it/s]

In [ ]:
question_mark = ['siapa', 'siapakah',
                    'apa', 'apakah', 'adakah',
                    'dimana', 'dimanakah', 'darimanakah',
                    'kapan', 'kapankah',
                    'bagaimana', 'bagaimanakah',
                    'kenapa', 'mengapa',
                    'berapa', 'berapakah', 'seberapa']

In [ ]:
hipo = "kapankah musik hip hop kah pertama kali muncul?"

for i in hipo.split():
    if (i == "kah") and (i not in question_mark):
        print(i)

In [ ]:
def create_nli_dataframe(df, type='replace first'):
    
    nli_df = pd.DataFrame()
    
    question_mark = ['siapa', 'siapakah',
                    'apa', 'apakah', 'adakah',
                    'dimana', 'dimanakah', 'darimanakah',
                    'kapan', 'kapankah',
                    'bagaimana', 'bagaimanakah',
                    'kenapa', 'mengapa',
                    'berapa', 'berapakah', 'seberapa']
    
    if type == 'rule based':
        for i in tqdm(range(df.shape[0])):
            premise = df['Context'][i]
            hypothesis = df['Question'][i]
            
            hypothesis = hypothesis.replace('kah', '')
            
            for j in hypothesis.split():
                if j in question_mark:
                    if j == 'siapa' or j == 'siapakah':
                        pred_hypothesis = hypothesis.replace('?', '')
                        pred_hypothesis = pred_hypothesis.replace(j, '')
                        pred_hypothesis = f"{pred_hypothesis} adalah {df['Prediction Answer'][i]}"

                        gold_hypothesis = hypothesis.replace('?', '')
                        gold_hypothesis = gold_hypothesis.replace(j, '')
                        gold_hypothesis = f"{gold_hypothesis} adalah {df['Gold Answer'][i]}"
                    
                    elif j == 'apa' or j == 'apakah' or j == 'adakah':
                        pred_hypothesis = hypothesis.replace('?', '')
                        pred_hypothesis = pred_hypothesis.replace(j, '')
                        pred_hypothesis = f"{pred_hypothesis} adalah {df['Prediction Answer'][i]}"

                        gold_hypothesis = hypothesis.replace('?', '')
                        gold_hypothesis = gold_hypothesis.replace(j, '')
                        gold_hypothesis = f"{gold_hypothesis} adalah {df['Gold Answer'][i]}"
                        
                    elif j == 'dimana' or j == 'dimanakah':
                        pred_hypothesis = hypothesis.replace('?', '')
                        pred_hypothesis = pred_hypothesis.replace(j, '')
                        pred_hypothesis = f"{pred_hypothesis} di {df['Prediction Answer'][i]}"

                        gold_hypothesis = hypothesis.replace('?', '')
                        gold_hypothesis = gold_hypothesis.replace(j, '')
                        gold_hypothesis = f"{gold_hypothesis} di {df['Gold Answer'][i]}"
                    
                    elif j == 'darimanakah':
                        pred_hypothesis = hypothesis.replace('?', '')
                        pred_hypothesis = pred_hypothesis.replace(j, '')
                        pred_hypothesis = f"{pred_hypothesis} dari {df['Prediction Answer'][i]}"

                        gold_hypothesis = hypothesis.replace('?', '')
                        gold_hypothesis = gold_hypothesis.replace(j, '')
                        gold_hypothesis = f"{gold_hypothesis} dari {df['Gold Answer'][i]}"
                    
                    elif j == 'kapan' or j == 'kapankah':
                        pred_hypothesis = hypothesis.replace('?', '')
                        pred_hypothesis = pred_hypothesis.replace(j, '')
                        pred_hypothesis = f"{pred_hypothesis} pada {df['Prediction Answer'][i]}"

                        gold_hypothesis = hypothesis.replace('?', '')
                        gold_hypothesis = gold_hypothesis.replace(j, '')
                        gold_hypothesis = f"{gold_hypothesis} pada {df['Gold Answer'][i]}"

                    elif j == 'bagaimana' or j == 'bagaimanakah':
                        pred_hypothesis = hypothesis.replace('?', '')
                        pred_hypothesis = pred_hypothesis.replace(j, '')
                        pred_hypothesis = f"{pred_hypothesis} adalah {df['Prediction Answer'][i]}"

                        gold_hypothesis = hypothesis.replace('?', '')
                        gold_hypothesis = gold_hypothesis.replace(j, '')
                        gold_hypothesis = f"{gold_hypothesis} adalah {df['Gold Answer'][i]}"
                    
                    elif j == 'kenapa' or j == 'mengapa':
                        pred_hypothesis = hypothesis.replace('?', '')
                        pred_hypothesis = pred_hypothesis.replace(j, 'alasan')
                        pred_hypothesis = f"{pred_hypothesis} adalah karena {df['Prediction Answer'][i]}"

                        gold_hypothesis = hypothesis.replace('?', '')
                        gold_hypothesis = gold_hypothesis.replace(j, 'alasan')
                        gold_hypothesis = f"{gold_hypothesis} adalah karena {df['Gold Answer'][i]}"
                    
                    elif j == 'berapa' or j == 'berapakah' or j == 'seberapa': 
                        pred_hypothesis = hypothesis.replace('?', '')
                        pred_hypothesis = pred_hypothesis.replace(j, '')
                        pred_hypothesis = f"{pred_hypothesis} adalah {df['Prediction Answer'][i]}"

                        gold_hypothesis = hypothesis.replace('?', '')
                        gold_hypothesis = gold_hypothesis.replace(j, '')
                        gold_hypothesis = f"{gold_hypothesis} adalah {df['Gold Answer'][i]}"
                    
            nli_df = nli_df.append({'Premise': premise,
                           'Prediction Hypothesis': pred_hypothesis,
                            'Gold Hypothesis': gold_hypothesis
                           }, ignore_index=True)

    elif type == 'replace first':
        for i in tqdm(range(df.shape[0])):
            premise = df['Context'][i]
            hypothesis = df['Question'][i]
            pred_hypothesis = hypothesis.replace('?', '')
            pred_hypothesis = pred_hypothesis.replace(hypothesis.split()[0], 
                                            df['Prediction Answer'][i])

            gold_hypothesis = hypothesis.replace('?', '')
            gold_hypothesis = gold_hypothesis.replace(hypothesis.split()[0], 
                                            df['Gold Answer'][i])
            
            nli_df = nli_df.append({'Premise': premise,
                               'Prediction Hypothesis': pred_hypothesis,
                                'Gold Hypothesis': gold_hypothesis
                               }, ignore_index=True)
    
    elif type == 'replace question mark':
        for i in tqdm(range(df.shape[0])):
            premise = df['Context'][i]
            hypothesis = df['Question'][i]
            for j in hypothesis.split():
                if j in question_mark:
                    pred_hypothesis = hypothesis.replace('?', '')
                    pred_hypothesis = pred_hypothesis.replace(j, df['Prediction Answer'][i])
                    
                    gold_hypothesis = hypothesis.replace('?', '')
                    gold_hypothesis = gold_hypothesis.replace(j, df['Gold Answer'][i])
                
            nli_df = nli_df.append({'Premise': premise,
                                   'Prediction Hypothesis': pred_hypothesis,
                                    'Gold Hypothesis': gold_hypothesis
                                   }, ignore_index=True)
    
    elif type == 'machine generation': pass # TODO
    
    elif type == 'add adalah': 
        for i in tqdm(range(df.shape[0])):
            premise = df['Context'][i]
            hypothesis = df['Question'][i]
            pred_hypothesis = hypothesis.replace('?', '')
            pred_hypothesis = pred_hypothesis.replace(hypothesis.split()[0], '')
            pred_hypothesis = f"{pred_hypothesis} adalah {df['Prediction Answer'][i]}"

            gold_hypothesis = hypothesis.replace('?', '')
            gold_hypothesis = gold_hypothesis.replace(hypothesis.split()[0], '')
            gold_hypothesis = f"{gold_hypothesis} adalah {df['Gold Answer'][i]}"
            
            nli_df = nli_df.append({'Premise': premise,
                               'Prediction Hypothesis': pred_hypothesis,
                                'Gold Hypothesis': gold_hypothesis
                               }, ignore_index=True)
    
    elif type == 'just concat answer and question':
        for i in tqdm(range(df.shape[0])):
            premise = df['Context'][i]
            hypothesis = df['Question'][i]

            pred_hypothesis = f"{hypothesis} {df['Prediction Answer'][i]}"         
            gold_hypothesis = f"{hypothesis} {df['Gold Answer'][i]}"
                
            nli_df = nli_df.append({'Premise': premise,
                                   'Prediction Hypothesis': pred_hypothesis,
                                    'Gold Hypothesis': gold_hypothesis
                                   }, ignore_index=True)
    '''
    kata_tanya = []
    for i in tqdm(range(df.shape[0])): 
        hypothesis = df['Question'][i]
        kata_tanya.append(hypothesis.split()[0])
    unik = set(np.unique(kata_tanya))
    temp3 = [x for x in unik if x not in question_mark]
    print(temp3)
    '''
        
    return nli_df

In [ ]:
nli_df = create_nli_dataframe(qas_df, type='rule based')
nli_df

## Mencoba cara retrieve model dari HF

In [ ]:
from transformers import pipeline

pretrained_name_sc = "afaji/fine-tuned-IndoNLI-Augmented-with-indobert-base-uncased"
pretrained_name_qa = "afaji/fine-tuned-DatasetQAS-TYDI-QA-ID-with-indobert-base-uncased-with-ITTL-without-freeze-LR-1e-05"
tokenizer_kwargs = {'padding': True, 'truncation': True, 'max_length': 512}

nlp_sc = pipeline(task="text-classification", model=pretrained_name_sc, tokenizer=pretrained_name_sc)
nlp_qa = pipeline(task="question-answering", model=pretrained_name_qa, tokenizer=pretrained_name_qa)

In [ ]:
x = nlp_sc({'text': nli_df['Premise'][0], 'text_pair': nli_df['Prediction Hypothesis'][0]}, top_k=3)
x

In [ ]:
nlp_qa(question=qas_df['Question'][1], context=qas_df['Context'][1], device=device)

## Coba buat method evaluasi

In [ ]:
def compute_after_postprocess(df=nli_df):
    for i in tqdm(range(df.shape[0])):
        predicted = nlp_sc({'text': df['Premise'][i], 'text_pair': df['Prediction Hypothesis'][i]}, 
                           top_k=1, **tokenizer_kwargs)[0]['label']
        print(predicted)

In [ ]:
predictions_idx = np.argsort(predict_result.predictions, axis=2)[:, :, 1 * -1]
predictions_idx

In [ ]:
def smoothing(predict_result, type='regular'):
    pass

In [ ]:
def evaluation(predict_result, type_smoothing="replace first", type_qas="TODO"):
    
    # 1. Ekstrak dari QAS
    DEFAULT_INDEX_LARGEST = 1
    predictions_idx = np.argsort(predict_result.predictions, axis=2)[:, :, DEFAULT_INDEX_LARGEST * -1]
    label_array = np.asarray(predict_result.label_ids)
    question_decoded = []
    context_decoded = []
    pred_answer_decoded = []
    gold_answer_decoded = []
    
    question_d = None
    context_d = None
    
    for i in tqdm(range(len(predict_result.predictions[0]))):
        start_pred_idx = predictions_idx[0][i]
        end_pred_idx = predictions_idx[1][i] + 1
        pred_answer = tokenizer.decode(tokenized_data_qas_id_validation[i]['input_ids']
                                       [start_pred_idx: end_pred_idx], skip_special_tokens=True)
        #pred_answer_decoded.append(pred_answer)
        
        start_gold_idx = label_array[0][i]
        end_gold_idx = label_array[1][i] + 1
        gold_answer = tokenizer.decode(tokenized_data_qas_id_validation[i]['input_ids']
                                       [start_gold_idx: end_gold_idx], skip_special_tokens=True)
        gold_answer_decoded.append(gold_answer)
        
        question = []
        context = []
        
        for j in range(len(tokenized_data_qas_id_validation[i]['token_type_ids'])):
            if tokenized_data_qas_id_validation[i]['token_type_ids'][j] == 0:
                question.append(tokenized_data_qas_id_validation[i]['input_ids'][j])
                question_d = tokenizer.decode(question, skip_special_tokens=True)
            else:
                context.append(tokenized_data_qas_id_validation[i]['input_ids'][j])
                context_d = tokenizer.decode(context, skip_special_tokens=True)
            
            # Smoothing (OTW)
            if type_smoothing == 'replace first':
                for k in tqdm(range(df.shape[0])):
                    premise = df['Context'][k]
                    hypothesis = df['Question'][k]
                    pred_hypothesis = hypothesis.replace('?', '')
                    pred_hypothesis = pred_hypothesis.replace(hypothesis.split()[0], 
                                                    df['Prediction Answer'][k])

                    gold_hypothesis = hypothesis.replace('?', '')
                    gold_hypothesis = gold_hypothesis.replace(hypothesis.split()[0], 
                                                    df['Gold Answer'][k])

                    nli_df = nli_df.append({'Premise': premise,
                                       'Prediction Hypothesis': pred_hypothesis,
                                        'Gold Hypothesis': gold_hypothesis
                                       }, ignore_index=True)

            elif type_smoothing == 'replace question mark':
                for k in tqdm(range(df.shape[0])):
                    premise = df['Context'][k]
                    hypothesis = df['Question'][k]
                    for l in hypothesis.split():
                        if j in question_mark:
                            pred_hypothesis = hypothesis.replace('?', '')
                            pred_hypothesis = pred_hypothesis.replace(l, df['Prediction Answer'][k])

                            gold_hypothesis = hypothesis.replace('?', '')
                            gold_hypothesis = gold_hypothesis.replace(l, df['Gold Answer'][k])
            
            # Cek entails
            #predicted = nlp_sc({'text': pred_answer, 'text_pair': context_d}, top_k=3, **tokenizer_kwargs)[0]['label'] 
            """
            if predicted == 'entailment':
                question_decoded.append(tokenizer.decode(question, skip_special_tokens=True))
                context_decoded.append(tokenizer.decode(context, skip_special_tokens=True))
                pred_answer_decoded.append(pred_answer)
                # Bisa jadi: break;
            
            # Bagian elif bisa di-comment untuk opsi kedua
            elif predicted == 'neutral':
                question_decoded.append(tokenizer.decode(question, skip_special_tokens=True))
                context_decoded.append(tokenizer.decode(context, skip_special_tokens=True))
                pred_answer_decoded.append(pred_answer)
            
            else:
                
                for index_largest in range(400 + 1):
                    
                    predictions_idx_inside_loop = np.argsort(predict_result.predictions, axis=2)[:, :, index_largest * -1]
                    start_pred_idx = predictions_idx_inside_loop[0][i]
                    end_pred_idx = predictions_idx_inside_loop[1][i] + 1
                    pred_answer_inside_loop = tokenizer.decode(tokenized_data_qas_id_validation[i]['input_ids']
                                                   [start_pred_idx: end_pred_idx], skip_special_tokens=True)
                    
                    predicted = nlp_sc({'text': pred_answer_inside_loop, 'text_pair': context_d}, 
                               top_k=3, **tokenizer_kwargs)[0]['label']
                    
                    if predicted == 'entailment':
                        question_decoded.append(tokenizer.decode(question, skip_special_tokens=True))
                        context_decoded.append(tokenizer.decode(context, skip_special_tokens=True))
                        pred_answer_decoded.append(pred_answer_inside_loop)
                        break
                    
                    else: continue
            """
        
    
    qas_df = pd.DataFrame({'Context': context_decoded, 
                           'Question': question_decoded, 
                           'Prediction Answer': pred_answer_decoded,
                          'Gold Answer': gold_answer_decoded})
    
    return qas_df
    
    
    
    # 2. Jalanin smoothing.
    #qas_df = create_qas_dataframe(predict_result, index_largest=1)
    #nli_df = create_nli_dataframe(qas_df, type='replace first')
    
    # 3. Cek entail, mau only entail, atau entail & neutral
    # 4. Return DataFrame QAS.
    #pass

In [ ]:
evaluation(predict_result)